<a href="https://colab.research.google.com/github/Zhuoyue-Huang/urop_2022_ml/blob/main/Try_and_prove.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
"""try and prove different results"""
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from tqdm import tqdm
import statistics
import numpy as np
import pandas as pd 
import scipy.stats as stats
from copy import deepcopy

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
# Linear auto-encoder model
class LAE(nn.Module):
    def __init__(self, n, p):
        super(LAE, self).__init__()
        self.n = n
        self.p = p
        self.w1 = nn.Linear(n, p, bias=False)
        self.w2 = nn.Linear(p, n, bias=False)

    def forward(self, y):
        y = self.w1(y)
        y = self.w2(y)
        return y

## Masked lienar auto-encoder model
# Define different types of masks
def mask_basic(prob, sample_num, x_dim, cst=0, device='cuda'):
    return torch.zeros(sample_num, x_dim, device=device).bernoulli_(prob)*(1-cst) + cst

def mask_probs(prob_list: torch.Tensor, sample_num, x_dim, cst=0, device='cuda'):
    return torch.zeros(sample_num, x_dim, device=device).bernoulli_(prob_list)*(1-cst) + cst

# def mask_patches_block(prob, patch_size: torch.Tensor, sample_num, sample_dim: torch.Tensor, device='cuda'):
#     x_dim = sample_dim[0]*sample_dim[1]
#     div_check = sample_dim % patch_size == torch.zeros(2)
#     if torch.all(div_check):
#         pix_num = torch.div(sample_dim, patch_size, rounding_mode='floor')
#         mat_patches = torch.zeros(sample_num, *pix_num).bernoulli_(prob).to(device)
#         mat_patches = torch.repeat_interleave(mat_patches, patch_size[1], dim=2)
#         return mat_patches.repeat_interleave(patch_size[0], dim=1).view(sample_num, x_dim)
#     else:
#         raise NotImplementedError(f"Both height ({sample_dim[0]}) and width ({sample_dim[1]}) should be divisible by patch_size ({patch_size}).")

def mask_patches_plain(prob, patch_size, sample_num, x_dim, cst=0, device='cuda'):
    if not x_dim % patch_size:
        pix_num = x_dim // patch_size
        mat_patches = torch.zeros(sample_num, pix_num).bernoulli_(prob)
        return mat_patches.repeat_interleave(patch_size, dim=1).to(device)*(1-cst) + cst
    else:
        raise NotImplementedError

# Masked autoencoder (linear)
class M_LAE(nn.Module):
    def __init__(self, prob, x_dim, f_dim, type='basic', patch_size=None, sample_dim=None, cst=0, mask_val=0):
        super(M_LAE, self).__init__()
        self.prob = prob
        if sample_dim is not None:
            self.sample_dim = sample_dim
            self.H, self.W = sample_dim
            self.n = self.H * self.W
        self.n = x_dim
        self.p = f_dim
        self.cst = cst
        self.mask_val = mask_val
        if type not in ['basic', 'probs', 'patches']:
            raise NotImplementedError("Could only implement 'basic', 'probs' and 'patches' type of masking.")
        else:
            self.masking_type = type
        if patch_size is not None:
            self.patch_size = patch_size
        w1 = nn.Linear(self.n, self.p, bias=False)
        w2 = nn.Linear(self.p, self.n, bias=False)
        self.body = nn.Sequential(*[w1, w2])
    
    def forward(self, X, mask=None):
        m = X.shape[0]
        if mask is None:
            if self.masking_type == 'basic':
                mask = mask_basic(self.prob, m, self.n, cst=self.cst)
            elif self.masking_type == 'probs':
                mask = mask_probs(self.prob, m, self.n, cst=self.cst)
            elif self.masking_type == 'patches':
                mask = mask_patches_plain(self.prob, self.patch_size, m, self.n, cst=self.cst)
        Y = mask*X + self.mask_val*(1-mask)
        Y = self.body(Y)
        return Y

class FE_Net(nn.Module):
    def __init__(self, f_dim, y_dim):
        super(FE_Net, self).__init__()
        self.theta = nn.Linear(f_dim, y_dim, bias=False)

    def forward(self, W):
        return self.theta(W)

# representation learning
def repr_learning(data_dict, model_parameters, criterion, type, epochs=200, device='cuda'):
    X_train, y_train, _, _, X_test, y_test = data_dict.values()

    params = list(model_parameters)
    W1 = params[0].clone().detach()
    f_train = X_train @ W1.T
    f_test = X_test @ W1.T
    f_dim = f_train.shape[1]
    y_dim = y_train.shape[1]

    if type=='ls':
        theta = (torch.inverse(f_train.T @ f_train) @ f_train.T @ y_train).T
        loss = criterion(y_test, f_test @ theta.T)
        return loss.item()
    elif type=='gd':
        data_dict_fe = {'X_train': f_train, 'y_train': y_train,
                        'X_test': f_test, 'y_test': y_test}
        net_fe = FE_Net(f_dim, y_dim).to(device)
        theta = list(net_fe.parameters())
        optimizer = optim.Adam(theta, lr=0.0001)
        ### TRAINING ###
        train_loop(data_dict_fe, net_fe, criterion, optimizer, epochs=epochs, record=False, type='fe')
        loss_fe = test_loop(data_dict_fe, net_fe, criterion, type='fe')
        return loss_fe

### Examine the effect of detach

In [ ]:
# Linear auto-encoder model
class LAE(nn.Module):
    def __init__(self, n, p):
        super(LAE, self).__init__()
        self.n = n
        self.p = p
        self.w1 = nn.Linear(n, p, bias=False)
        self.w2 = nn.Linear(p, n, bias=False)

    def forward(self, y):
        y = self.w1(y)
        y = self.w2(y)
        return y

class FE_Net(nn.Module):
    def __init__(self, in_dim, out_dim):
        super(FE_Net, self).__init__()
        self.in_dim = in_dim
        self.out_dim = out_dim
        self.theta = nn.Linear(in_dim, out_dim, bias=False)

    def forward(self, W):
        return self.theta(W)

In [ ]:
def train_loop(data_dict, model, criterion, optimizer, type, epochs=10, sample_average=10, record=True):
    train_loss = []
    val_loss = []
    if type=='encoder':
        train_inputs = data_dict['train_inputs']
        train_targets = train_inputs
    elif type=='fe':
        train_inputs = data_dict['train_inputs']
        train_targets = data_dict['train_targets']
        sample_average = 1

    for epoch in range(epochs+1):
        loss_total = 0
        optimizer.zero_grad()
        for i in range(sample_average):
            train_outputs = model(train_inputs)
            loss = criterion(train_outputs, train_targets)
            loss_total += loss
        loss_total /= sample_average
        loss_total.backward()
        optimizer.step()
        if record:
          train_loss.append(loss_total.item())
          if epochs>=5 and epoch%(epochs//5)==0:
              v_loss = test_loop(data_dict, model, criterion, type)
              val_loss.append(v_loss)
              print('epoch: ', epoch, ', train loss: ', loss.item(), ', val loss', v_loss)
    if record:
        return {'train_loss': train_loss, 'val_loss': val_loss}
    else:
        return loss_total.item()

def test_loop(data_dict, model, criterion, type):
    if type=='encoder':
        val_inputs = data_dict['val_inputs']
        val_targets = val_inputs
    elif type=='fe':
        val_inputs = data_dict['val_inputs']
        val_targets = data_dict['val_targets']

    with torch.no_grad():
        val_outputs = model(val_inputs)
        loss = criterion(val_outputs, val_targets)
    return loss.item()

In [ ]:
# feature extraction
def feature_extraction_test(data_dict, model_parameters, criterion, type, epochs=200, device='cuda'):
    train_inputs, train_targets, val_inputs, val_targets = data_dict.values()

    params = list(model_parameters)
    W1 = params[0]
    W10 = deepcopy(W1)
    W10 = W10.clone().detach()
    print(W10)
    train_inputs_fe0 = train_inputs @ W10.T
    val_inputs_fe0 = val_inputs @ W10.T

    W11 = deepcopy(W1)
    W11 = W11.clone()
    print(W11)
    train_inputs_fe1 = (train_inputs @ W11.T).detach()
    val_inputs_fe1 = (val_inputs @ W11.T).detach()

    reduction_dim = train_inputs_fe1.shape[1]
    target_dim = train_targets.shape[1]

    if type=='ls':
        param_fe0 = (torch.inverse(train_inputs_fe0.T@train_inputs_fe0) @ train_inputs_fe0.T @ train_targets).T
        loss0 = criterion(val_targets, val_inputs_fe0 @ param_fe0.T)
        print('test0', loss0.item())

        param_fe1 = (torch.inverse(train_inputs_fe1.T@train_inputs_fe1) @ train_inputs_fe1.T @ train_targets).T
        loss1 = criterion(val_targets, val_inputs_fe1 @ param_fe1.T)
        print('test1', loss1.item())
        #return loss.item()
    elif type=='gd':
        net_fe = FE_Net(reduction_dim, target_dim).to(device)
        net_fe0 = deepcopy(net_fe)
        net_fe1 = deepcopy(net_fe)

        data_dict_fe0 = {'train_inputs': train_inputs_fe0, 'train_targets': train_targets,
                        'val_inputs': val_inputs_fe0, 'val_targets': val_targets}
        param_fe0 = list(net_fe0.parameters())
        optimizer0 = optim.Adam(param_fe0, lr=0.0001)
        ### TRAINING ###
        loss_fe0 = train_loop(data_dict_fe0, net_fe0, criterion, optimizer0, epochs=epochs, record=False, type='fe')
        print('test0', loss_fe0)
        data_dict_fe1 = {'train_inputs': train_inputs_fe1, 'train_targets': train_targets,
                        'val_inputs': val_inputs_fe1, 'val_targets': val_targets}
        param_fe1 = list(net_fe1.parameters())
        optimizer1 = optim.Adam(param_fe1, lr=0.0001)
        ### TRAINING ###
        loss_fe1 = train_loop(data_dict_fe1, net_fe1, criterion, optimizer1, epochs=epochs, record=False, type='fe')
        print('test1', loss_fe1)
        #return loss_fe

In [ ]:
train_num = 50
val_num = 20
H = 8
W = 8
sample_dim = torch.tensor([H, W])
feature_num = H * W
reduction_dim = feature_num
target_dim = feature_num

prob = 0.75
prob_list = torch.rand(feature_num)*0.2 + 0.65
patch_size = torch.div(sample_dim, 2, rounding_mode='floor')

train_inputs = torch.rand(train_num, feature_num) * 2
train_inputs = train_inputs.to(device)
train_targets = torch.rand(train_num, target_dim)
train_targets = train_targets.to(device)
val_inputs = torch.rand(val_num, feature_num) * 2
val_inputs = val_inputs.to(device)
val_targets = torch.rand(val_num, target_dim)
val_targets = val_targets.to(device)
data_dict = {'train_inputs': train_inputs, 'train_targets': train_targets,
             'val_inputs': val_inputs, 'val_targets': val_targets}

In [ ]:
net_LAE = LAE(feature_num, reduction_dim).to(device)

params = list(net_LAE.parameters())
criterion = nn.MSELoss()
optimizer = optim.Adam(params, lr=0.001)

### TRAINING ###
train_loop(data_dict, net_LAE, criterion, optimizer, epochs=500, record=False, type='encoder')

# feature extraction
loss_ls = feature_extraction_test(data_dict, net_LAE.parameters(), criterion, type='ls', epochs=500)
loss_gd = feature_extraction_test(data_dict, net_LAE.parameters(), criterion, type='gd', epochs=500)

tensor([[-0.1336,  0.0542,  0.0287,  ..., -0.1842,  0.0315,  0.1009],
        [-0.0978, -0.0365, -0.1108,  ..., -0.0541, -0.1404,  0.0120],
        [ 0.1117,  0.0700,  0.0037,  ...,  0.0655,  0.0070, -0.1101],
        ...,
        [ 0.1055,  0.0057, -0.0793,  ..., -0.0329,  0.1920,  0.0874],
        [-0.1235, -0.0956,  0.1405,  ..., -0.0077,  0.0426, -0.1306],
        [ 0.1496, -0.1518, -0.1922,  ..., -0.0546, -0.0099, -0.0903]],
       device='cuda:0')
tensor([[-0.1336,  0.0542,  0.0287,  ..., -0.1842,  0.0315,  0.1009],
        [-0.0978, -0.0365, -0.1108,  ..., -0.0541, -0.1404,  0.0120],
        [ 0.1117,  0.0700,  0.0037,  ...,  0.0655,  0.0070, -0.1101],
        ...,
        [ 0.1055,  0.0057, -0.0793,  ..., -0.0329,  0.1920,  0.0874],
        [-0.1235, -0.0956,  0.1405,  ..., -0.0077,  0.0426, -0.1306],
        [ 0.1496, -0.1518, -0.1922,  ..., -0.0546, -0.0099, -0.0903]],
       device='cuda:0', grad_fn=<CloneBackward0>)
test0 606.8356323242188
test1 606.8356323242188
tensor([[-

### Least square vs gradient decent

##### check whether least square estimator has zero gradient

In [ ]:
def train_loop(data_dict, model, criterion, optimizer, type, epochs=10, sample_average=10, record=True):
    train_loss = []
    val_loss = []
    if type=='encoder':
        train_x = data_dict['train_x']
        train_y = train_x
    elif type=='fe':
        train_x = data_dict['train_x']
        train_y = data_dict['train_y']
        sample_average = 1

    for epoch in range(epochs+1):
        loss_total = 0
        optimizer.zero_grad()
        for i in range(sample_average):
            outputs = model(train_x)
            loss = criterion(outputs, train_y)
            loss_total += loss
        loss_total /= sample_average
        loss_total.backward()
        optimizer.step()
        if record:
          train_loss.append(loss_total.item())
          if epochs>=5 and epoch%(epochs//5)==0:
              v_loss = test_loop(data_dict, model, criterion, type)
              val_loss.append(v_loss)
              print('epoch: ', epoch, ', train loss: ', loss.item(), ', val loss', v_loss)
    if record:
        return {'train_loss': train_loss, 'val_loss': val_loss}
    else:
        return loss_total.item()

def test_loop(data_dict, model, criterion, type):
    if type=='encoder':
        val_x = data_dict['val_x']
        val_y = val_x
    elif type=='fe':
        val_x = data_dict['val_x']
        val_y = data_dict['val_y']

    with torch.no_grad():
        outputs = model(val_x)
        loss = criterion(outputs, val_y)
    return loss.item()

In [ ]:
class FE_Net(nn.Module):
    def __init__(self, z_dim, y_dim):
        super(FE_Net, self).__init__()
        self.theta = nn.Linear(z_dim, y_dim, bias=False)

    def forward(self, W):
        return self.theta(W)

def feature_extraction(data_dict, model_parameters, criterion, type, epochs=200, device='cuda'):
    train_x, train_y, val_x, val_y, _, _ = data_dict.values()

    params = list(model_parameters)
    W1 = params[0].clone().detach()
    train_z = train_x @ W1.T
    val_z = val_x @ W1.T
    z_dim = train_z.shape[1]
    y_dim = train_y.shape[1]

    if type=='ls':
        print(torch.linalg.matrix_rank(train_z))
        theta = (torch.inverse(train_z.T @ train_z) @ train_z.T @ train_y).T
        print(-train_z.T @ train_y + train_z.T @ train_z @ theta.T)
        loss = criterion(val_y, val_z @ theta.T)
        return loss.item()
    elif type=='gd':
        data_dict_fe = {'train_x': train_z, 'train_y': train_y,
                        'val_x': val_z, 'val_y': val_y}
        net_fe = FE_Net(z_dim, y_dim).to(device)
        theta = list(net_fe.parameters())
        optimizer = optim.Adam(theta, lr=0.00001)
        ### TRAINING ###
        loss_fe = train_loop(data_dict_fe, net_fe, criterion, optimizer, epochs=epochs, record=False, type='fe')
        return loss_fe

In [ ]:
train_num, val_num, test_num = (60, 20, 20)
sample_num_split = (train_num, train_num+val_num)
total = train_num+val_num+test_num

# need to consider x_dim < and > z_dim
z_dim = 10 # dimension of z
H = 4
W = 4
sample_dim = torch.tensor([H, W])
x_dim = H * W
y_dim = z_dim // 2

prob = 0.75
prob_list = torch.rand(x_dim)*0.2 + 0.65
patch_size = torch.tensor([2, 2])

In [ ]:
z = torch.normal(mean=0, std=1, size=(total,z_dim)) # here distribution is high dimensional guassian
z = z.to(device)

U = torch.rand(x_dim, z_dim)
U = U.to(device)
V = torch.rand(y_dim, z_dim)
V = V.to(device)

x = z @ U.T
y = z @ V.T
train_x, val_x, test_x = torch.tensor_split(x, sample_num_split)
train_y, val_y, test_y = torch.tensor_split(y, sample_num_split)
data_dict = {'train_x': train_x, 'train_y': train_y, 'val_x': val_x, 'val_y': val_y, 'test_x': test_x, 'test_y': test_y}
fe_loss_dict = {'LAE': [], 'MLAE_basic': [], 'MLAE_probs': [], 'MLAE_patches': []}

In [ ]:
learning_rate = 0.01
epochs = 500
epochs_fe = 250

In [ ]:
net_LAE = LAE(x_dim, z_dim).to(device)

params = list(net_LAE.parameters())
criterion = nn.MSELoss()
optimizer = optim.Adam(params, lr=learning_rate)

### TRAINING ###
train_loop(data_dict, net_LAE, criterion, optimizer, epochs=epochs, record=False, type='encoder')

# feature extraction
loss_ls = feature_extraction(data_dict, net_LAE.parameters(), criterion, type='ls')
loss_gd = feature_extraction(data_dict, net_LAE.parameters(), criterion, type='gd', epochs=50000)
print('least square loss', loss_ls)
print('gradient decent loss', loss_gd)

tensor(10, device='cuda:0')
tensor([[ 5.3024e-04,  6.4087e-04,  6.7139e-04,  8.3160e-04,  6.7902e-04],
        [ 3.3569e-04,  3.6621e-04,  2.4414e-04,  5.1880e-04,  4.5776e-04],
        [ 8.0109e-05,  1.1253e-04,  1.2589e-04,  1.4019e-04,  1.2922e-04],
        [ 1.0681e-04,  1.6022e-04,  1.9836e-04,  1.8311e-04,  1.3733e-04],
        [-4.5776e-05, -3.4332e-05, -3.0518e-05, -6.8665e-05, -3.8147e-05],
        [-2.4796e-04, -2.7466e-04, -2.4414e-04, -3.9673e-04, -3.3569e-04],
        [ 1.1253e-04,  1.6022e-04,  1.5831e-04,  2.0981e-04,  1.7548e-04],
        [ 4.5776e-05,  3.0518e-05, -7.6294e-05,  9.1553e-05,  7.6294e-05],
        [ 1.9073e-05,  2.2888e-05,  3.0518e-05,  2.4319e-05,  2.2888e-05],
        [-2.0599e-04, -2.7466e-04, -1.9836e-04, -3.8147e-04, -3.2043e-04]],
       device='cuda:0')
least square loss 5.977891098796206e-11
gradient decent loss 0.09737139940261841


##### set gradient of network to the theoretical sol

In [ ]:
n = 10
p = 5
m = 100
X = torch.rand(m, n)
y = torch.rand(m, p)
theta = (torch.inverse(X.T @ X) @ X.T @ y).T

net = FE_Net(n, p)
net.theta.weight.data = theta
#print(theta)
#print(list(net.parameters()))

In [ ]:
criterion = nn.MSELoss()
params = list(net.parameters())
optimiser = optim.SGD(params, lr=1)
optimiser.zero_grad()
output = net(X)
loss = criterion(output, y)
loss.backward()
optimiser.step()

In [ ]:
print(list(net.parameters())[0].clone().detach()-theta)

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])


### check the analytic solution of autoencoder
Linear autoencoder

In [33]:
train_num, val_num, test_num = (100, 500, 2000) # larger number of dataset
sample_num_split = (train_num, train_num + val_num)
total = train_num+val_num+test_num

# need to consider f_dim <, =, > z_dim
z_dim = 8 # dimension of z
H = 3
W = 3
sample_dim = torch.tensor([H, W])
x_dim = H * W
f_dim = 8
y_dim = z_dim // 2

z = torch.normal(mean=0, std=1, size=(total,z_dim), generator=torch.random.manual_seed(522)) # here distribution is high dimensional guassian
z = z.to(device)

U = torch.rand(x_dim, z_dim, generator=torch.random.manual_seed(522))
U = U.to(device)
V = torch.rand(y_dim, z_dim, generator=torch.random.manual_seed(522))
V = V.to(device)

x = z @ U.T
y = z @ V.T
X_train, X_val, X_test = torch.tensor_split(x, sample_num_split)
y_train, y_val, y_test = torch.tensor_split(y, sample_num_split)
data_dict = {'X_train': X_train, 'y_train': y_train, 'X_val': X_val, 'y_val': y_val, 'X_test': X_test, 'y_test': y_test}

In [ ]:
Sigma = X_train.T @ X_train
U, S, Vh = torch.linalg.svd(Sigma)
U = U[:, :f_dim]
W1 = U.T
W2 = U

In [ ]:
net = LAE(x_dim, f_dim)
net.w1.weight.data = W1
net.w2.weight.data = W2

In [ ]:
criterion = nn.MSELoss()
params = list(net.parameters())
optimiser = optim.SGD(params, lr=0.3)

optimiser.zero_grad()
output = net(X_train)
loss = criterion(output, X_train)
loss.backward()
optimiser.step()

In [ ]:
print(list(net.parameters())[0].clone().detach()-W1)

tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]], device='cuda:0')


In [ ]:
print(list(net.parameters())[1].clone().detach()-W2)

tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]], device='cuda:0')


Masked linear autoencoder

In [120]:
cst = 0
mask_val = -1
prob = 0.75
prob_list = torch.rand(x_dim)
patch_size = 3
model_type = 'MLAE_patches'
X_train, _, X_val, _, _, _ = data_dict.values()
X = X_train
mask_dict = {}
if model_type == 'MLAE_basic':
    first_moment = prob + cst * (1-prob)
    second_moment = prob + cst**2 * (1-prob)
    second_moment_reverse = 1-prob + cst**2 * prob
    mask_dict['mean'] = (torch.ones(train_num, x_dim, device=device) * first_moment)
    mask_dict['square'] = torch.ones(x_dim, x_dim, device=device) * first_moment**2
    mask_dict['square'].fill_diagonal_(second_moment)

    x0_sum = torch.sum(X, 0, keepdim=True).repeat(x_dim, 1)
    mask_x = torch.diag(torch.ones(x_dim, device=device))
    mean_reverse = mask_x*(first_moment-second_moment)+(1-mask_x)*first_moment*(1-first_moment)
    square_reverse = torch.ones(x_dim, x_dim, device=device) * (1-first_moment)**2
    square_reverse.fill_diagonal_(second_moment_reverse)
    mask_dict['var_increment'] = mask_val*(x0_sum+x0_sum.T)*mean_reverse + (mask_val**2)*train_num*square_reverse

elif model_type == 'MLAE_probs':
    first_moment = prob_list + cst * (1-prob_list)
    second_moment = prob_list + cst**2 * (1-prob_list)
    second_moment_reverse = 1-prob_list + cst**2*prob_list
    mask_dict['mean'] = first_moment.repeat(train_num, 1).to(device)
    mask_dict['square'] = first_moment.view(x_dim, 1) @ first_moment.view(1, x_dim)
    mask_dict['square'] = (mask_dict['square'].fill_diagonal_(0) + torch.diag(second_moment)).to(device)
    
    x0_sum = torch.sum(X, 0, keepdim=True).repeat(x_dim, 1)
    p_mat = first_moment.repeat(x_dim, 1).to(device)
    mask_x = torch.diag(torch.ones(x_dim, device=device))
    mean_reverse = torch.diag(first_moment-second_moment).to(device)+(1-mask_x)*(1-p_mat.T)*p_mat
    square_reverse = (1-first_moment).view(x_dim, 1) @ (1-first_moment).view(1, x_dim)
    square_reverse = (square_reverse.fill_diagonal_(0) + torch.diag(second_moment_reverse)).to(device)
    mask_dict['var_increment'] = mask_val*mean_reverse*x0_sum + mask_val*mean_reverse.T*x0_sum.T*mask_x + (mask_val**2)*train_num*square_reverse
    
elif model_type == 'MLAE_patches':
    first_moment = prob + cst * (1-prob)
    second_moment = prob + cst**2 * (1-prob)
    second_moment_reverse = 1-prob + cst**2 * prob
    mask_dict['mean'] = torch.ones(train_num, x_dim, device=device) * first_moment
    # plain patches
    patch_block_mat = torch.ones(patch_size, patch_size, device=device)
    mask_dict['square'] = torch.block_diag(*[patch_block_mat]*(x_dim//patch_size))*(second_moment-first_moment**2) + first_moment**2

    x0_sum = torch.sum(X, 0, keepdim=True).repeat(x_dim, 1)
    mask_x = torch.block_diag(*[patch_block_mat]*(x_dim//patch_size))
    mean_reverse = mask_x*(first_moment-second_moment) + (1-mask_x)*first_moment*(1-first_moment)
    square_reverse = torch.block_diag(*[patch_block_mat]*(x_dim//patch_size))*(second_moment_reverse-(1-first_moment)**2) + (1-first_moment)**2
    mask_dict['var_increment'] = mask_val*(x0_sum+x0_sum.T)*mean_reverse + (mask_val**2)*train_num*square_reverse
# block patches
# patch_size = [2, 2]
# pix_num = torch.div(torch.tensor(sample_dim), torch.tensor(patch_size), rounding_mode='floor')
# mat_patches = torch.arange(pix_num[0]*pix_num[1]).view(*pix_num)
# mat_patches = torch.repeat_interleave(mat_patches, patch_size[1], dim=1)
# mat_patches = torch.repeat_interleave(mat_patches, patch_size[0], dim=0).view(n)
# mask_dict['MLAE_patches']['square'] = torch.zeros(x_dim, x_dim)
# for i in range(x_dim):
#     for j in range(x_dim):
#         if mat_patches[i] == mat_patches[j]:
#             mask_dict['MLAE_patches']['square'][i, j] = prob
#         else:
#             mask_dict['MLAE_patches']['square'][i, j] = prob**2

In [121]:
try:
    inv = torch.inverse(mask_dict['square']*(X.T@X) + mask_dict['var_increment'])
except torch._C._LinAlgError:
    # Sigma_XX = X.T@X
    # U_XX, S_XX, Vh_XX = torch.linalg.svd(Sigma_XX)
    # print(S_XX)
    # inv = Vh_XX.T*torch.inverse(torch.diag(S_XX+1e-4))*U_XX.T

    perturbation = torch.diag(torch.ones(x_dim, device=device)*1e-20)
    inv = torch.inverse(mask_dict['square']*(X.T@X) + mask_dict['var_increment'] + perturbation)
Sigma = X.T@(mask_dict['mean']*X+(1-mask_dict['mean'])*mask_val) @ inv @ ((mask_dict['mean']*X+(1-mask_dict['mean'])*mask_val).T@X)
U, _, _ = torch.linalg.svd(Sigma)
U = U[:, :f_dim]
W1 = U.T @ (X.T@(mask_dict['mean']*X+(1-mask_dict['mean'])*mask_val)) @ inv
W2 = U
# if model_type == 'MLAE_basic':
#     loss = criterion((mask_basic(prob, val_num, x_dim, cst=cst)*X_val)@W1.T@W2.T, X_val)
# elif model_type == 'MLAE_probs':
#     loss = criterion((mask_probs(prob_list, val_num, x_dim, cst=cst)*X_val)@W1.T@W2.T, X_val)
# elif model_type == 'MLAE_patches':
#     loss = criterion((mask_patches_plain(prob, patch_size, val_num, x_dim, cst=cst)*X_val)@W1.T@W2.T, X_val)
W1

tensor([[ 0.3622,  0.3539,  0.3769,  0.6519,  0.2496,  0.2082,  0.2046,  0.3284,
          0.5619],
        [ 0.3591, -0.5932,  0.1454,  0.1847,  0.3201, -0.5561, -0.6692,  0.1349,
          0.3569],
        [ 0.4806,  0.1729, -0.6253, -0.6148,  0.7439,  0.0240, -0.0637,  0.1269,
         -0.0108],
        [-0.0820, -0.2700,  0.1755, -0.4858, -0.0740,  0.6628, -0.0432, -0.1605,
          0.3196],
        [ 0.5569, -0.2725, -0.4557,  0.0931, -0.2152,  0.1204,  0.0846,  0.3601,
         -0.2899],
        [ 0.1341, -0.0817, -0.1302,  0.0890,  0.0767, -0.1459,  0.3233, -0.6347,
          0.4165],
        [-0.2469, -0.0327,  0.1990,  0.0063,  0.1312, -0.1045,  0.0174,  0.1893,
         -0.1381],
        [-0.0048,  0.0226, -0.0220,  0.0208, -0.0132, -0.0050, -0.0187, -0.0025,
          0.0200]], device='cuda:0')

In [122]:
rand_loop = 500
total_W1 = 0
total_W2 = 0
for i in range(rand_loop):
    net = M_LAE(prob, x_dim, f_dim, patch_size=patch_size, type='patches', cst=cst, mask_val=mask_val).to(device)
    net.body[0].weight.data = W1.clone().detach()
    net.body[1].weight.data = W2.clone().detach()
    criterion = nn.MSELoss()
    optimiser = optim.SGD(net.body.parameters(), lr=0.0003)

    optimiser.zero_grad()
    output = net(X)
    loss = criterion(output, X)
    loss.backward()
    optimiser.step()

    total_W1 += net.body[0].weight.data.clone().detach().cpu()-W1.cpu()
    total_W2 += net.body[1].weight.data.clone().detach().cpu()-W2.cpu()

total_W1 /= rand_loop
total_W2 /= rand_loop

In [123]:
print(total_W1)

tensor([[-8.5276e-07,  1.8799e-07, -7.8166e-07, -1.2444e-06, -6.2627e-07,
         -1.3762e-06,  1.3942e-06,  1.5778e-06,  1.5960e-06],
        [ 3.5554e-07,  4.9472e-07,  4.7800e-07,  3.9661e-07,  3.8475e-07,
          3.3772e-07,  2.6131e-07,  3.4001e-07,  2.9844e-07],
        [ 1.6022e-07,  3.8660e-07,  9.0003e-08,  5.9009e-08, -1.6689e-09,
          3.4884e-08,  3.4384e-07,  3.4049e-07,  2.3138e-07],
        [-2.9206e-08,  8.5533e-08, -7.1377e-08,  1.4544e-07,  8.3283e-08,
          1.3840e-07,  1.2475e-07,  1.2651e-07,  1.6481e-07],
        [ 1.6582e-07,  7.0333e-08,  1.7148e-07,  1.8397e-07,  1.1426e-07,
          2.7882e-07,  1.0145e-07,  1.2279e-08,  1.7166e-08],
        [-2.3007e-08, -6.5401e-08, -4.9919e-08, -2.1753e-07, -1.9665e-07,
         -1.4737e-07, -3.4511e-08, -3.2783e-08, -4.8280e-08],
        [ 1.6388e-07,  1.6355e-07,  1.5143e-07, -1.0855e-07, -1.2553e-07,
         -9.7424e-08,  1.9938e-08,  7.5370e-08,  6.2257e-08],
        [ 7.0622e-08,  3.5275e-08,  7.0363e-08, 

In [124]:
print(total_W2)

tensor([[ 3.3611e-07, -1.7625e-07,  1.8887e-07, -6.4850e-08, -6.3866e-08,
          4.7117e-08,  1.0371e-07,  4.3288e-09],
        [-5.5063e-07, -6.2406e-08,  1.6105e-07, -1.1069e-07,  3.5167e-09,
          3.9190e-08,  6.2786e-08,  6.0201e-09],
        [-1.2946e-07, -8.8811e-09,  1.9693e-07, -9.6381e-08, -1.0407e-07,
          4.2960e-08,  1.0434e-07,  7.3910e-09],
        [-4.0305e-07,  3.3677e-08,  1.0777e-07,  1.2100e-08, -3.9071e-08,
          5.0098e-08,  6.1095e-08,  7.1526e-10],
        [ 1.5104e-07, -1.9947e-07,  1.7130e-07, -1.2354e-07, -1.2755e-08,
          2.9206e-09,  5.5015e-08,  6.8843e-09],
        [-4.6635e-07, -6.2346e-08,  1.3968e-07, -9.4175e-09,  9.3430e-09,
          9.6440e-08,  5.1141e-08,  8.0094e-09],
        [-7.3507e-07, -7.0930e-08,  9.2626e-08,  8.6427e-09,  5.3763e-08,
          5.0068e-08,  2.1491e-08,  2.5034e-09],
        [ 7.5072e-07, -2.7063e-07,  1.5861e-07, -7.2330e-08, -3.7432e-08,
          3.1233e-08,  6.0916e-08,  6.4820e-09],
        [ 5.6392